In [1]:
# Imports
from sqlalchemy import create_engine, inspect, Table
import pandas as pd
import numpy as np
import json
import country_converter as coco
import matplotlib.pyplot as plt

# Some pandas display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 500)

# Loading owid data to sql database

In [2]:
# Open a connection to the postgresql database
db_conn = create_engine("postgresql://student:infomdss@db_dashboard:5432/covid")

In [ ]:
owid = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv", parse_dates=['date'], dayfirst = True, sep=',', skip_blank_lines = False)

In [ ]:
#Input format: csv in which each row is a unique date, country pair and columns contain a lot of covid-related...
#...info for the given country on the given date.
#Output format: a sql table in which again each row is a unique date, country pair,but only The Netherlands,...
#... Italy and Japan, the countries of our interest, are included and only the columns containing information...,
#...relevant to our analysis is kept.

db_conn.execute("DROP TABLE IF EXISTS owid CASCADE;")
owid = owid[(owid.location == 'Italy') | (owid.location=='Japan') | (owid.location == 'Netherlands')]
toDrop = ['continent','total_deaths','new_deaths','new_deaths_smoothed','total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million','total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'stringency_index',
       'population', 'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities','life_expectancy',
       'human_development_index', 'excess_mortality_cumulative_absolute',
       'excess_mortality_cumulative', 'excess_mortality',
       'excess_mortality_cumulative_per_million']
toDrop = [x for x in toDrop if x in owid.columns]
owid.drop(toDrop, axis = 1, inplace=True)
owid.to_sql("owid", db_conn, if_exists='replace')
df = pd.read_sql_table('owid', db_conn)
df

In [ ]:
pd.unique(owid['iso_code'])

In [ ]:
db_conn.execute("DROP VIEW IF EXISTS owidWY CASCADE;")
query = '''
CREATE OR REPLACE VIEW owidWY AS 
SELECT *,
CAST(EXTRACT(WEEK FROM date) AS int) AS week,
CAST(EXTRACT(YEAR FROM date) AS int) AS year
FROM owid
'''
db_conn.execute(query)
df = pd.read_sql('SELECT * FROM owidWY',db_conn)
df 

# Loading covariants data to sql

In [ ]:
#input format: json in which first level objects represent countries. Each country has attributes for..
#...weekly dates during the pandemic, the total number of analysed covid infection sequences during that week...
#...and attributes for number of sequences that were of a given variant during that week.
#output format: a sql table where each row is a unique country, date pair and there are columns for...
#... country_iso, date total sequences analysed and number of all sequences found. Only data for NLD, ITA...
#... and JPN is kept.

covarRaw = pd.read_json('https://raw.githubusercontent.com/hodcroftlab/covariants/07b0ea786205ad7269c09d46d2f7b13ef01b667e/cluster_tables/EUClusters_data.json')
dfs = []

for i in range(len(covarRaw.index)):
    country = covarRaw.index[i]
    if not country in ['Netherlands','Italy','Japan']:
        continue
    dtn = covarRaw['countries'][i]
    dtn['iso_code'] = coco.convert(country)
    df = pd.DataFrame.from_dict(dtn)
    df['date'] = pd.to_datetime(df['week'])
    df.drop('week',axis=1,inplace=True)
    dfs.append(df)
    
covarRaw = None
covar = pd.concat(dfs)


# Getting percentages instead of total

In [ ]:
cols = list(covar)
#put the date and iso column to the front of the list
cols.insert(1, cols.pop(cols.index('date')))
cols.insert(2, cols.pop(cols.index('iso_code')))

covar = covar.loc[:, cols]

#create a column with the unclassified varients
covar['other/unknown_variants']= covar['total_sequences'] - covar.iloc[:,3:].sum(axis=1)

#instead of absulute valies we calculte 
columns_to_divide_by = covar.columns.values[3:]
covar.iloc[:,3:] = covar[columns_to_divide_by].div(covar.total_sequences, axis=0)

covar.tail(5)

In [ ]:
db_conn.execute("DROP TABLE IF EXISTS covar CASCADE;")
covar.to_sql("covar", db_conn, if_exists='replace')

# Loading event cancel data to sql

In [ ]:
#input format: csv in which rows consist of unique countries, columns consist of dates. Each entry... 
# ...represents the cancelation level for the given date in the given country
#output format: a sql table where each row is a unique country date pair, columns for country_iso (from our country selection of NLD,... 
# ...ITA and JPN), date and event cancellation level

eventsRaw = pd.read_csv("https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/c4m_restrictions_on_gatherings.csv", sep=',', skip_blank_lines = False)
eventsRaw.drop(['Unnamed: 0', 'country_name','region_code','region_name','jurisdiction'],axis=1,inplace=True)

dfs= []
 
for iso in ['NLD','JPN','ITA']:
    df = eventsRaw[eventsRaw['country_code']==iso]
    df.drop('country_code',axis=1,inplace=True)
    df= df.T
    df.columns = ['event_cancellation_level']
    df['date'] = pd.to_datetime(df.index)
    df['iso_code'] = iso
    dfs.append(df)

eventsRaw = None
events = pd.concat(dfs)
events
    

In [ ]:
db_conn.execute("DROP TABLE IF EXISTS events CASCADE;")
events.to_sql("events", db_conn, if_exists='replace')

# Joining all relevant data in single sql view

In [ ]:
#Joining variant data to owid data
db_conn.execute("DROP VIEW IF EXISTS owidcovar CASCADE;")
query = '''
CREATE OR REPLACE VIEW owidcovar AS 
SELECT 
    owidWY.*, "total_sequences", "20I (Alpha, V1)", "20H (Beta, V2)", "20J (Gamma, V3)", "21A (Delta)", 
    "21I (Delta)", "21J (Delta)", "21K (Omicron)", "21L (Omicron)", "22A (Omicron)", "22B (Omicron)", "22C (Omicron)", 
    "22D (Omicron)", "21B (Kappa)", "21D (Eta)", "21F (Iota)", "21G (Lambda)", "21H (Mu)", "20B/S:732A", "20E (EU1)", 
    "21C (Epsilon)", "20A/S:439K", "S:677H.Robin1", "20A.EU2", "20A/S:98F", "20B/S:626S", "20B/S:1122L", "20A/S:126A", "20C/S:80Y", "other/unknown_variants"
FROM owidWY
LEFT JOIN covar ON (owidWY.date = covar.date AND owidWY.iso_code = covar.iso_code);
'''
db_conn.execute(query)
df = pd.read_sql('SELECT * FROM owidcovar',db_conn)
# df.head()

In [ ]:
#Joining event cancellation data to owid and variant data
db_conn.execute("DROP VIEW IF EXISTS all_relevant CASCADE;")
query = '''
CREATE OR REPLACE VIEW all_relevant AS 
SELECT 
    owidcovar.*, event_cancellation_level
FROM owidcovar
LEFT JOIN events ON (owidcovar.date = events.date AND owidcovar.iso_code = events.iso_code);
'''
db_conn.execute(query)
df = pd.read_sql('SELECT * FROM all_relevant',db_conn)
df.head()

# Creating an interpolated df for analysis

In [ ]:
#input the original df and the the columns that you want to be interpolated.
#returns a the original df with interpolated columns
def interpolation(df, columns_to_be_interpolated):
    
    
    #we only interpolated between values not before or after, therefor the limit_direction is both
    #we want to our interpolate limit to be two weeks
    remaining_columns = [elem for elem in df.columns.values.tolist() if elem not in columns_to_be_interpolated]
    remaining_columns_df = df[remaining_columns] 
    df_to_be_interpolated = df[columns_to_be_interpolated]
    
    df = df_to_be_interpolated.interpolate(method ='linear', limit_direction ='both', limit = 20)

    df = pd.concat([remaining_columns_df, df], axis=1)
    
    return df


In [ ]:
JPN_df = df.loc[df['iso_code'] == 'JPN']
columns_to_be_interpolated_JPN = ['icu_patients', 'icu_patients_per_million', 'hosp_patients', 'hosp_patients_per_million', 
"total_sequences", "20I (Alpha, V1)", "20H (Beta, V2)", "20J (Gamma, V3)", "21A (Delta)", 
"21I (Delta)", "21J (Delta)", "21K (Omicron)", "21L (Omicron)", "22A (Omicron)", "22B (Omicron)", "22C (Omicron)", 
"22D (Omicron)", "21B (Kappa)", "21D (Eta)", "21F (Iota)", "21G (Lambda)", "21H (Mu)", "20B/S:732A", "20E (EU1)", 
"21C (Epsilon)", "20A/S:439K", "S:677H.Robin1", "20A.EU2", "20A/S:98F", "20B/S:626S", "20B/S:1122L", "20A/S:126A", "20C/S:80Y", "other/unknown_variants"]

JPN_df_interpolated = interpolation(JPN_df, columns_to_be_interpolated_JPN)

ITA_df = df.loc[df['iso_code'] == 'ITA']
ITA_df_interpolated = interpolation(ITA_df, columns_to_be_interpolated_JPN[5:])

columns = ['total_vaccinations_per_hundred','people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred','total_boosters_per_hundred', 
           'weekly_icu_admissions', 'weekly_icu_admissions_per_million', 'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million',
           'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated','total_boosters']

NLD_df = df.loc[df['iso_code'] == 'NLD']

NLD_df_interpolated = interpolation(NLD_df, columns + columns_to_be_interpolated_JPN)

df_interpolated = pd.concat([JPN_df_interpolated, ITA_df_interpolated, NLD_df_interpolated], ignore_index=True)

NLD_df_interpolated.tail()

In [ ]:
#pushing the interpolated df to SQL
db_conn.execute("DROP TABLE IF EXISTS df_interpolated CASCADE;")
df_interpolated.to_sql("df_interpolated", db_conn, if_exists='replace')

# Creating views for success factors

In [ ]:
#getting the relevant kpi data for cases (excluding variant distribution for now)
db_conn.execute("DROP VIEW IF EXISTS cases CASCADE;")
query = '''
CREATE OR REPLACE VIEW cases AS 
SELECT iso_code, date,new_cases_per_million, new_cases_smoothed_per_million, reproduction_rate
FROM all_relevant
'''
db_conn.execute(query)
df = pd.read_sql('SELECT * FROM cases',db_conn)
df

In [ ]:
#getting the relevant kpi data for cases (including variant distribution)
db_conn.execute("DROP VIEW IF EXISTS casesplus CASCADE;")
query = '''
CREATE OR REPLACE VIEW casesplus AS 
SELECT iso_code, date,new_cases_per_million, new_cases_smoothed_per_million, reproduction_rate,
    "total_sequences", "20I (Alpha, V1)", "20H (Beta, V2)", "20J (Gamma, V3)", "21A (Delta)", 
    "21I (Delta)", "21J (Delta)", "21K (Omicron)", "21L (Omicron)", "22A (Omicron)", "22B (Omicron)", "22C (Omicron)", 
    "22D (Omicron)", "21B (Kappa)", "21D (Eta)", "21F (Iota)", "21G (Lambda)", "21H (Mu)", "20B/S:732A", "20E (EU1)", 
    "21C (Epsilon)", "20A/S:439K", "S:677H.Robin1", "20A.EU2", "20A/S:98F", "20B/S:626S", "20B/S:1122L", "20A/S:126A", "20C/S:80Y"
FROM all_relevant
'''
db_conn.execute(query)
df = pd.read_sql('SELECT * FROM casesplus',db_conn)
df

In [ ]:
#getting the relevant kpi data for vaccinations
query = '''
CREATE OR REPLACE VIEW vaccinations AS 
SELECT iso_code, date, people_fully_vaccinated_per_hundred, total_boosters_per_hundred,new_vaccinations_smoothed_per_million
FROM all_relevant
'''
db_conn.execute(query)
df =pd.read_sql('SELECT * FROM vaccinations',db_conn)
df

In [ ]:
#getting the relevant kpi data for hospitalizations
db_conn.execute("DROP VIEW IF EXISTS hospitalizations, weeklyHos CASCADE;")
query = '''
CREATE OR REPLACE VIEW hospitalizations AS 
SELECT all_relevant.iso_code, date, icu_patients_per_million, weekly_icu_admissions_per_million, hosp_patients_per_million
FROM all_relevant
ORDER BY all_relevant.iso_code ASC, date ASC
'''
db_conn.execute(query)
df = pd.read_sql('SELECT * FROM hospitalizations',db_conn)
df

# Plotting the data simply to increase data understanding

In [ ]:
def simplePlot(db_conn,isoCode, factor, lastN=-1):
    df = pd.read_sql('SELECT * FROM ' + factor,db_conn)
    df = df[df.iso_code==isoCode]
    if lastN > 0:
        df = df.tail(lastN)
    kpis = [x for x in df.columns if not x in ['iso_code','date'] ]
    for kpi in kpis:
        print(kpi)
        plt.plot(df.index,kpi,data=df)
        plt.title(kpi + ' in ' + isoCode)
        plt.xticks(rotation=90)
        plt.show()

In [ ]:
lastN = 2000
simplePlot(db_conn,'NLD','hospitalizations',lastN)
simplePlot(db_conn,'ITA','hospitalizations',lastN)
simplePlot(db_conn,'JPN','hospitalizations',lastN)

In [ ]:
lastN = 1000
simplePlot(db_conn,'NLD','cases',lastN)
simplePlot(db_conn,'ITA','cases',lastN)
simplePlot(db_conn,'JPN','cases',lastN)

In [ ]:
lastN = 200
simplePlot(db_conn,'NLD','vaccinations',lastN)
simplePlot(db_conn,'ITA','vaccinations',lastN)
simplePlot(db_conn,'JPN','vaccinations',lastN)